# Backup/Sync custom directory to OneDrive using onedrive-client 

Environment: 
- OS: Ubuntu 22.04

## Description
I want to backup/sync some custom directories to OneDrive using the onedrive-client on Ubuntu 22.04.
In this configuration, i setup 1 way sync *upload only* from local to OneDrive.
And using *bind* mount to map custom directory(`/home/nii/...`) to onedrive-client sync directory (`~/OneDrive`).


## 1. Install onedrive-client - the lastest version


In [ ]:
wget -qO - https://download.opensuse.org/repositories/home:/npreining:/debian-ubuntu-onedrive/xUbuntu_24.04/Release.key \
| sudo gpg --dearmor -o /usr/share/keyrings/onedrive-archive-keyring.gpg

In [ ]:
echo "deb [signed-by=/usr/share/keyrings/onedrive-archive-keyring.gpg] \
https://download.opensuse.org/repositories/home:/npreining:/debian-ubuntu-onedrive/xUbuntu_24.04/ ./" \
| sudo tee /etc/apt/sources.list.d/onedrive.list

In [ ]:
sudo apt update
sudo apt install onedrive

Using the cmd above to install the lastest version of onedrive-client from the official PPA repository.
And then, check the installed version. You should see the version is `2.5.x` or higher.
```
$ onedrive --version
```
The output should be similar to:
```
onedrive v2.5.9-1+np1+1.1

```

## 2. Create selective sync configuration directory and setup onedrive config
Create the configuration directory for onedrive-client selective sync.

In [ ]:
nano ~/.config/onedrive/sync_list

Then, paste the following content into the `sync_list` file. Adjust the paths according to your needs.
```bash
Ubuntu_Desktops/
Ubuntu_Documents/
Ubuntu_Downloads/
Ubuntu_Pictures/
Ubuntu_Music/
Ubuntu_Videos/
```

In [ ]:
nano ~/.config/onedrive/config

Then paste the following content into the `config` file:
```bash
sync_dir = "~/OneDrive"

# Skip file/dir
skip_dir = ".git,node_modules,.venv,__pycache__"
skip_file = "*.tmp,*.log"

# Config: upload only (1 way sync)
upload_only = "true"

# Delete file at local:
# no_remote_delete = "true" : keep file at Cloud OnDrive when deleted at local
no_remote_delete = "true"
```


## 3. Create some directories in OneDrive root for bind mount
Create the following directories in your OneDrive root directory (`~/OneDrive/`):
```bash
Ubuntu_Desktops
Ubuntu_Documents        
Ubuntu_Downloads
Ubuntu_Pictures
Ubuntu_Music
Ubuntu_Videos
```

## 4. Setup bind mount for custom directories

Example, to bind mount `/home/nii/Desktop` to `~/OneDrive/Ubuntu_Desktops`, use the following command:

In [ ]:
sudo mount --bind /home/nii/Desktop ~/OneDrive/Ubuntu_Desktops

Set up automatic bind mount at boot by adding entries to `/etc/fstab`. Open the file with a text editor with sudo privileges:



In [ ]:
sudo nano /etc/fstab

Add the following lines at the end of the file:
```bash
/home/nii/Desktop    /home/nii/OneDrive/Ubuntu_Desktops    none    bind    0    0
/home/nii/Documents  /home/nii/OneDrive/Ubuntu_Documents   none    bind    0    0
/home/nii/Downloads  /home/nii/OneDrive/Ubuntu_Downloads   none    bind    0    0
/home/nii/Pictures   /home/nii/OneDrive/Ubuntu_Pictures    none    bind    0    0
/home/nii/Music      /home/nii/OneDrive/Ubuntu_Music       none    bind    0    0
/home/nii/Videos     /home/nii/OneDrive/Ubuntu_Videos      none    bind    0    0
```

Save the file and exit the text editor. The bind mounts will be applied automatically at the next system boot.

## 5. Start onedrive-client sync
Now, you can start the onedrive-client sync process by running:

In [ ]:
onedrive --synchronize --resync

## 6. Active the onedrive-client as a systemd service
To ensure that the onedrive-client runs automatically in the background and starts at boot, you can set it up as a systemd service.

In [ ]:
systemctl --user enable onedrive
systemctl --user start onedrive

Check the status of the onedrive-client service to ensure it is running correctly. You can watch log in real-time using:

In [ ]:
journalctl --user -xeu onedrive -f

For test: you create/update,... one file, and it will appear in current terminal which is running the above command.

---

### The end 